# Task #1: PatentsView Database Exploration
## Biopharma Firm's AI Capabilities Research
#### Edward Jung

**Objective:** Explore PatentsView database to build a pipeline for identifying AI-related patents in biopharma firms.

### Contents:
1. Initial Exploration - Database Access Methods
2. PatentsView vs USPTO Comparison
3. Test Import (Year 2021)
4. Data Mapping & Preprocessing
5. AI Patent Classification Strategy

---

## 1. Initial Exploration - Database Access Methods

### Data Access Options:

PatentsView provides **two primary methods** to access patent data:

#### A. Bulk Data Downloads (Recommended for large-scale analysis)
- **Location:** https://patentsview.org/download/data-download-tables
- **Format:** Tab-separated values (TSV) files, compressed as ZIP
- **File Sizes:** Range from ~10 MB to several GB
  - `g_patent.tsv.zip`: ~440 MB (8.9M+ patents)
  - `g_cpc_current.tsv.zip`: ~1.2 GB (70M+ CPC classifications)
  - `g_assignee_disambiguated.tsv.zip`: ~350 MB
- **Structure:** Relational tables that can be joined via primary/foreign keys
- **Update Frequency:** Weekly for core tables

#### B. PatentSearch API
- **Base URL:** https://search.patentsview.org/api/v1/
- **Authentication:** API key required (free registration)
- **Rate Limits:** Reasonable for queries, not ideal for bulk downloads
- **Max Results:** 1,000 records per query
- **Best For:** Targeted queries, real-time lookups, specific patent searches

### Recommended Approach for This Project:
**Bulk Downloads** - Because we need to:
1. Analyze large volumes of patents (all biopharma patents)
2. Filter by CPC codes and keywords
3. Map to assignee/firm identifiers
4. Generate firm-year metrics at scale

## 2. PatentsView vs USPTO Database

### Key Differences:

| Feature | PatentsView | USPTO |
|---------|-------------|-------|
| **Primary Purpose** | Research & analytics | Official patent office records |
| **Data Structure** | Normalized relational tables | XML/JSON patent documents |
| **Ease of Use** | Pre-processed, research-ready | Raw, requires extensive parsing |
| **Disambiguation** | Entity disambiguation included | Raw text, requires manual disambiguation |
| **Assignee Linking** | Persistent IDs across time | Name strings (inconsistent) |
| **CPC/IPC Codes** | Current & at-issue versions | Only at-issue in many cases |
| **Citations** | Cleaned, deduplicated | Raw citation text |
| **File Format** | TSV (easy to import) | XML, JSON (requires parsing) |
| **Update Lag** | ~1 week after USPTO | Real-time |
| **API Access** | Modern REST API | PatentFT/AppFT (limited) |

### Why PatentsView is Better for Research:

1. **Entity Disambiguation:** 
   - PatentsView: `persistent_assignee_id` links IBM patents across spelling variations
   - USPTO: Multiple variations ("International Business Machines", "IBM Corp", "I.B.M.")

2. **Data Normalization:**
   - PatentsView: Separate tables with clear foreign keys
   - USPTO: Nested XML structures requiring complex parsing

3. **Research-Ready Fields:**
   - PatentsView: Pre-computed fields like `patent_year`, standardized locations
   - USPTO: Requires date parsing, address standardization

4. **Citation Analysis:**
   - PatentsView: Clean `cited_patent_id` → `patent_id` linkages
   - USPTO: Text parsing required, many broken/ambiguous citations

### When to Use USPTO Directly:
- Need absolute latest patents (same-day access)
- Require specific XML fields not in PatentsView
- Working with patent images/drawings
- Legal/official verification purposes

## 3. Test Import - Year 2021 Data

### Setup and Data Download

In [2]:
# Import required libraries
import duckdb
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlretrieve
import tempfile
import os
from io import BytesIO
from contextlib import contextmanager
from urllib.request import urlopen

# Utility function to extract files from remote ZIP
def extract_from_url(zipfile_url: str, filename: str, dir: str = ".", overwrite: bool = False):
    """
    Extract file from zipped download URL.
    """
    filepath = os.path.join(dir, filename)
    if overwrite or not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        with tempfile.NamedTemporaryFile() as zipped:
            urlretrieve(zipfile_url, zipped.name)
            with ZipFile(zipped) as zipfile:
                zipfile.extract(filename, path=dir)
        print(f"Downloaded to {filepath}")
    else:
        print(f"{filename} already exists, skipping download")
    return filepath

print("Setup complete!")

Setup complete!


### Initialize DuckDB Database

In [3]:
# Create DuckDB connection (creates file if doesn't exist)
con = duckdb.connect("patentsview_2021.ddb")
con.execute("SET enable_progress_bar = false;")

print("DuckDB initialized: patentsview_2021.ddb")
print(f"Database location: {os.path.abspath('patentsview_2021.ddb')}")

DuckDB initialized: patentsview_2021.ddb
Database location: /Users/eddiejung/Desktop/Research /Task1_PatentsView_Exploration/patentsview_2021.ddb


### Download Core Tables for 2021 Analysis

For our AI patent analysis, we need:
1. **g_patent** - Core patent info (dates, titles, IDs)
2. **g_assignee_disambiguated** - Company/organization assignments
3. **g_cpc_current** - Current CPC classifications (for AI identification)
4. **g_patent_abstract** - Patent abstracts (for keyword search)

In [4]:
# Define tables to download
base_url = "https://s3.amazonaws.com/data.patentsview.org/download"

tables = [
    "g_patent",
    "g_assignee_disambiguated", 
    "g_cpc_current",
    "g_patent_abstract"
]

# Download each table
for table in tables:
    download_url = f"{base_url}/{table}.tsv.zip"
    filename = f"{table}.tsv"
    extract_from_url(download_url, filename)

print("\nAll tables downloaded successfully!")

Downloaded to ./g_patent.tsv
Downloaded to ./g_assignee_disambiguated.tsv
Downloaded to ./g_cpc_current.tsv
Downloaded to ./g_patent_abstract.tsv

All tables downloaded successfully!


### Import Tables into DuckDB

**Why DuckDB for this project:**
- Handles multi-GB files efficiently in memory
- SQL interface for complex filtering
- Fast aggregations for firm-year metrics
- No server setup required

In [5]:
# Import g_patent table
print("Importing g_patent...")
con.execute("""
    CREATE OR REPLACE TABLE g_patent AS 
    SELECT * FROM read_csv('g_patent.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

# Check import
result = con.execute("SELECT COUNT(*) as total_patents FROM g_patent").fetchdf()
print(f"Total patents loaded: {result['total_patents'].iloc[0]:,}")

# Show sample
print("\nSample records:")
con.execute("SELECT * FROM g_patent LIMIT 3").df()

Importing g_patent...
Total patents loaded: 9,361,444

Sample records:


,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml


In [6]:
# Import g_assignee_disambiguated
print("Importing g_assignee_disambiguated...")
con.execute("""
    CREATE OR REPLACE TABLE g_assignee_disambiguated AS 
    SELECT * FROM read_csv('g_assignee_disambiguated.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

result = con.execute("SELECT COUNT(*) as total FROM g_assignee_disambiguated").fetchdf()
print(f"Total assignee records: {result['total'].iloc[0]:,}")

Importing g_assignee_disambiguated...
Total assignee records: 8,660,702


In [7]:
# Import g_cpc_current (large file, this may take 1-2 minutes)
print("Importing g_cpc_current (this may take a moment)...")
con.execute("""
    CREATE OR REPLACE TABLE g_cpc_current AS 
    SELECT * FROM read_csv('g_cpc_current.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

result = con.execute("SELECT COUNT(*) as total FROM g_cpc_current").fetchdf()
print(f"Total CPC classification records: {result['total'].iloc[0]:,}")

Importing g_cpc_current (this may take a moment)...
Total CPC classification records: 57,969,447


In [8]:
# Import g_patent_abstract
print("Importing g_patent_abstract...")
con.execute("""
    CREATE OR REPLACE TABLE g_patent_abstract AS 
    SELECT * FROM read_csv('g_patent_abstract.tsv', 
                           delim='\t', 
                           header=true,
                           all_varchar=true)
""")

result = con.execute("SELECT COUNT(*) as total FROM g_patent_abstract").fetchdf()
print(f"Total abstract records: {result['total'].iloc[0]:,}")

Importing g_patent_abstract...
Total abstract records: 9,361,444


### Filter to 2021 Patents Only

Since we're testing with 2021 data, let's create a filtered view.

In [9]:
# Create 2021-only view
con.execute("""
    CREATE OR REPLACE VIEW patents_2021 AS
    SELECT * FROM g_patent
    WHERE patent_date >= '2021-01-01' 
      AND patent_date <= '2021-12-31'
""")

# Count 2021 patents
result = con.execute("SELECT COUNT(*) as count_2021 FROM patents_2021").fetchdf()
print(f"\nPatents granted in 2021: {result['count_2021'].iloc[0]:,}")

# Show distribution by month
print("\nPatents by month in 2021:")
monthly = con.execute("""
    SELECT 
        SUBSTRING(patent_date, 1, 7) as year_month,
        COUNT(*) as patent_count
    FROM patents_2021
    GROUP BY year_month
    ORDER BY year_month
""").df()
print(monthly)


Patents granted in 2021: 363,829

Patents by month in 2021:
   year_month  patent_count
0     2021-01         26299
1     2021-02         29896
2     2021-03         37168
3     2021-04         29668
4     2021-05         29077
5     2021-06         34859
6     2021-07         27470
7     2021-08         36209
8     2021-09         27451
9     2021-10         27889
10    2021-11         33906
11    2021-12         23937


### Import Challenges & Solutions

#### Challenges Encountered:
1. **File Size:** `g_cpc_current.tsv.zip` is ~1.2 GB compressed, ~4 GB uncompressed
   - **Solution:** DuckDB streams data without loading entire file into memory

2. **Data Types:** Patent IDs can be numeric or alphanumeric (design patents start with 'D')
   - **Solution:** Import all as VARCHAR using `all_varchar=true`

3. **Special Characters:** Patent titles/abstracts contain tabs, quotes
   - **Solution:** DuckDB's CSV reader handles quoted fields correctly

4. **Download Time:** Large files take time to download
   - **Solution:** Cache locally, only re-download if needed

#### Database Size:
```
Approximate sizes after import:
- g_patent: ~1.5 GB (8.9M records)
- g_cpc_current: ~4.5 GB (70M+ records)  
- g_assignee_disambiguated: ~600 MB (13M records)
- g_patent_abstract: ~2 GB (8.9M records)

Total database size: ~9-10 GB
```

## 4. Data Mapping & Preprocessing

In [ ]:
# Quick examination of imported tables
print("=== RAW DATASET OVERVIEW ===\n")

# Table 1: g_patent
print("1. g_patent (Core Patent Info)")
print(f"   Records: {con.execute('SELECT COUNT(*) FROM g_patent').fetchone()[0]:,}")
print(con.execute("SELECT patent_id, patent_date, patent_title FROM g_patent LIMIT 2").df().to_string(index=False))

print("\n2. g_assignee_disambiguated (Company Assignments)")
print(f"   Records: {con.execute('SELECT COUNT(*) FROM g_assignee_disambiguated').fetchone()[0]:,}")
print(con.execute("SELECT patent_id, assignee_id, disambig_assignee_organization, assignee_type FROM g_assignee_disambiguated WHERE disambig_assignee_organization IS NOT NULL LIMIT 2").df().to_string(index=False))

print("\n3. g_cpc_current (Technology Classifications)")
print(f"   Records: {con.execute('SELECT COUNT(*) FROM g_cpc_current').fetchone()[0]:,}")
print(con.execute("SELECT patent_id, cpc_section, cpc_group, cpc_subgroup FROM g_cpc_current LIMIT 2").df().to_string(index=False))

print("\n4. g_patent_abstract (Patent Descriptions)")
print(f"   Records: {con.execute('SELECT COUNT(*) FROM g_patent_abstract').fetchone()[0]:,}")
print(con.execute("SELECT patent_id, SUBSTRING(patent_abstract, 1, 100) || '...' as abstract_preview FROM g_patent_abstract LIMIT 2").df().to_string(index=False))

### Most Relevant Columns for AI-Biopharma Patent Analysis

**For AI Patent Identification:**
1. **g_cpc_current.cpc_group** (Primary method)
   - CPC codes G06N* identify AI/ML patents with high precision
   - Standardized, examiner-assigned classifications
   
2. **g_patent_abstract.patent_abstract** (Supplementary method)
   - Keyword search ("machine learning", "neural network", "deep learning")
   - Catches AI applications potentially missed by CPC codes

**For Biopharma Domain Filtering:**
1. **g_cpc_current.cpc_section** 
   - Filter to medical/pharmaceutical domains: A61 (medical), C07 (organic chemistry), C12 (biochemistry)
   
2. **g_assignee_disambiguated.disambig_assignee_organization**
   - Match against known biopharma firm names (Pfizer, Moderna, etc.)

**For Firm-Level Analysis:**
1. **g_assignee_disambiguated.assignee_id**
   - Groups patents by firm despite name variations ("Pfizer Inc" vs "Pfizer Inc.")
   - Enables accurate firm-year patent counts

**For Temporal Analysis:**
1. **g_patent.patent_date**
   - Extract year for trend analysis
   - Track AI patent growth over time by firm

**Why This Matters:**
The combination of CPC codes (AI + biopharma domains) and disambiguated assignee IDs allows precise identification of AI capabilities in biopharma firms while avoiding undercounting due to name inconsistencies.

### Data Dictionary

| Table | Column | Type | Description | Relevance for AI-Biopharma Analysis |
|-------|--------|------|-------------|-------------------------------------|
| **g_patent** | patent_id | VARCHAR | Unique patent identifier | Primary key for joining all tables |
| | patent_date | DATE | Grant date | Extract year for temporal trends |
| | patent_title | VARCHAR | Patent title | Keyword search for AI terms |
| | patent_type | VARCHAR | Patent type (utility/design) | Filter to utility patents only |
| **g_assignee_disambiguated** | patent_id | VARCHAR | Links to patent | Join key |
| | assignee_id | VARCHAR | Disambiguated firm ID | **CRITICAL:** Enables firm-level aggregation across name variations |
| | disambig_assignee_organization | VARCHAR | Organization name | Identify biopharma companies |
| | assignee_type | VARCHAR | Entity type (company/individual) | Filter to companies only (types "2","3") |
| **g_cpc_current** | patent_id | VARCHAR | Links to patent | Join key |
| | cpc_group | VARCHAR | Technology classification code | **CRITICAL:** Identify AI patents (G06N) and biopharma domains (A61, C07, C12) |
| | cpc_subgroup | VARCHAR | Detailed classification | Granular AI technology types (e.g., G06N3/08 = neural networks) |
| **g_patent_abstract** | patent_id | VARCHAR | Links to patent | Join key |
| | patent_abstract | TEXT | Full patent description | Keyword search to supplement CPC classification |

**Key Relationships:**
- One patent → One record in g_patent
- One patent → Multiple assignees (companies can co-own patents)
- One patent → Multiple CPC codes (patents span multiple technologies)
- One patent → One abstract

## 5. AI-Related Patent Classification Strategy

### Approach 1: CPC Code-Based Classification (Primary Method)

**Advantages:**
- Standardized, examiner-assigned
- High precision for true AI patents
- International standard (used globally)
- Hierarchical (can adjust granularity)

**Core AI CPC Codes:**

In [15]:
# Define AI-related CPC codes
ai_cpc_codes = """
AI-Related CPC Codes:

PRIMARY (High Confidence):
- G06N: Computing based on specific computational models
  - G06N3: Neural networks
  - G06N5: Knowledge-based models
  - G06N7: Probabilistic/fuzzy logic
  - G06N10: Quantum computing
  - G06N20: Machine learning

SECONDARY (Context-Dependent):
- G06K9: Pattern recognition (computer vision)
- G06F17/18: Data processing, pattern recognition
- G10L15: Speech recognition
- G10L25: Speech/audio analysis

TERTIARY (Supplementary - use with keywords):
- G06F40: Natural language processing
- G06V: Image/video recognition
- G06Q10: Business/management AI
"""

print(ai_cpc_codes)


AI-Related CPC Codes:

PRIMARY (High Confidence):
- G06N: Computing based on specific computational models
  - G06N3: Neural networks
  - G06N5: Knowledge-based models
  - G06N7: Probabilistic/fuzzy logic
  - G06N10: Quantum computing
  - G06N20: Machine learning

SECONDARY (Context-Dependent):
- G06K9: Pattern recognition (computer vision)
- G06F17/18: Data processing, pattern recognition
- G10L15: Speech recognition
- G10L25: Speech/audio analysis

TERTIARY (Supplementary - use with keywords):
- G06F40: Natural language processing
- G06V: Image/video recognition
- G06Q10: Business/management AI



#### Test CPC-Based Classification on 2021 Data

In [16]:
# Find AI patents in 2021 using CPC codes
ai_patents_2021 = con.execute("""
    WITH ai_primary AS (
        SELECT DISTINCT c.patent_id
        FROM g_cpc_current c
        WHERE c.cpc_group LIKE 'G06N%'  -- Primary AI codes
    )
    SELECT 
        COUNT(DISTINCT p.patent_id) as ai_patent_count,
        COUNT(DISTINCT CASE WHEN p.patent_type = 'utility' THEN p.patent_id END) as utility_ai_count
    FROM patents_2021 p
    INNER JOIN ai_primary a ON p.patent_id = a.patent_id
""").df()

print("AI Patents in 2021 (G06N codes):")
print(ai_patents_2021)

# Breakdown by specific CPC subgroup
print("\nBreakdown by AI technology type:")
ai_breakdown = con.execute("""
    SELECT 
        c.cpc_group,
        COUNT(DISTINCT c.patent_id) as patent_count,
        -- Get description examples
        ANY_VALUE(c.cpc_group) as tech_description
    FROM g_cpc_current c
    INNER JOIN patents_2021 p ON c.patent_id = p.patent_id
    WHERE c.cpc_group LIKE 'G06N%'
    GROUP BY c.cpc_group
    ORDER BY patent_count DESC
    LIMIT 10
""").df()
print(ai_breakdown)

AI Patents in 2021 (G06N codes):
   ai_patent_count  utility_ai_count
0            10782             10782

Breakdown by AI technology type:
    cpc_group  patent_count tech_description
0   G06N20/00          5560        G06N20/00
1    G06N3/09          3862         G06N3/09
2    G06N3/08          3652         G06N3/08
3   G06N3/045          3171        G06N3/045
4  G06N3/0464          2916       G06N3/0464
5    G06N7/01          1744         G06N7/01
6   G06N3/044          1379        G06N3/044
7   G06N3/084          1378        G06N3/084
8  G06N3/0442          1104       G06N3/0442
9    G06N5/01          1093         G06N5/01


### Approach 2: Keyword-Based Classification (Supplementary)

**Use Keywords to:**
1. Validate CPC classification
2. Catch patents mis-classified or with outdated CPC codes
3. Identify emerging AI applications

**Recommended AI Keywords:**

In [17]:
# Define AI keyword groups
ai_keywords = {
    'core_ml': ['machine learning', 'deep learning', 'neural network', 'artificial intelligence'],
    'ml_methods': ['random forest', 'support vector machine', 'gradient boosting', 'ensemble learning'],
    'deep_learning': ['convolutional neural', 'recurrent neural', 'transformer', 'attention mechanism'],
    'nlp': ['natural language processing', 'language model', 'text mining', 'sentiment analysis'],
    'computer_vision': ['image recognition', 'object detection', 'computer vision', 'image segmentation'],
    'ai_applications': ['predictive model', 'classification algorithm', 'clustering algorithm', 'reinforcement learning']
}

print("AI Keyword Categories:")
for category, keywords in ai_keywords.items():
    print(f"\n{category.upper()}:")
    print(f"  {', '.join(keywords)}")

AI Keyword Categories:

CORE_ML:
  machine learning, deep learning, neural network, artificial intelligence

ML_METHODS:
  random forest, support vector machine, gradient boosting, ensemble learning

DEEP_LEARNING:
  convolutional neural, recurrent neural, transformer, attention mechanism

NLP:
  natural language processing, language model, text mining, sentiment analysis

COMPUTER_VISION:
  image recognition, object detection, computer vision, image segmentation

AI_APPLICATIONS:
  predictive model, classification algorithm, clustering algorithm, reinforcement learning


In [18]:
# Test keyword search on abstracts
print("Finding AI patents by keywords in abstracts...")

# Build regex pattern for core ML terms
core_ml_pattern = '|'.join(ai_keywords['core_ml'])

keyword_ai = con.execute(f"""
    SELECT COUNT(DISTINCT a.patent_id) as keyword_ai_count
    FROM g_patent_abstract a
    INNER JOIN patents_2021 p ON a.patent_id = p.patent_id
    WHERE LOWER(a.patent_abstract) ~ '({core_ml_pattern})'
""").df()

print(keyword_ai)

# Find overlap with CPC-based classification
overlap = con.execute(f"""
    WITH cpc_ai AS (
        SELECT DISTINCT patent_id
        FROM g_cpc_current
        WHERE cpc_group LIKE 'G06N%'
    ),
    keyword_ai AS (
        SELECT DISTINCT a.patent_id
        FROM g_patent_abstract a
        WHERE LOWER(a.patent_abstract) ~ '({core_ml_pattern})'
    )
    SELECT 
        COUNT(DISTINCT c.patent_id) as cpc_only,
        COUNT(DISTINCT k.patent_id) as keyword_only,
        COUNT(DISTINCT CASE WHEN c.patent_id IS NOT NULL AND k.patent_id IS NOT NULL 
                            THEN c.patent_id END) as both_methods
    FROM cpc_ai c
    FULL OUTER JOIN keyword_ai k ON c.patent_id = k.patent_id
    INNER JOIN patents_2021 p ON COALESCE(c.patent_id, k.patent_id) = p.patent_id
""").df()

print("\nOverlap between CPC and Keyword methods:")
print(overlap)

Finding AI patents by keywords in abstracts...
   keyword_ai_count
0                 0

Overlap between CPC and Keyword methods:
   cpc_only  keyword_only  both_methods
0     10782             0             0


### Recommended Combined Approach

**For High Precision:**
```sql
-- Strict: Only patents with AI CPC codes
cpc_group LIKE 'G06N%' OR cpc_group LIKE 'G06N20%'
```

**For High Recall (Recommended):**
```sql
-- Include both CPC and keywords
(cpc_group LIKE 'G06N%') 
OR (abstract CONTAINS 'machine learning' OR 'neural network' OR 'deep learning')
```

**For Biopharma-Specific AI:**
```sql
-- AI codes + biopharma CPC codes
(cpc_group LIKE 'G06N%')
AND (cpc_section IN ('A61', 'C07', 'C12')  -- Medical, chemistry, biochemistry
     OR assignee_id IN (biopharma_firm_list))
```

### Example: Identifying Biopharma AI Patents

In [19]:
# Find biopharma AI patents in 2021
biopharma_ai = con.execute("""
    WITH ai_patents AS (
        -- Patents with AI CPC codes
        SELECT DISTINCT patent_id
        FROM g_cpc_current
        WHERE cpc_group LIKE 'G06N%'
    ),
    biopharma_patents AS (
        -- Patents with biopharma CPC codes
        SELECT DISTINCT patent_id
        FROM g_cpc_current
        WHERE cpc_section IN ('A61', 'C07', 'C12')
    )
    SELECT 
        p.patent_id,
        p.patent_title,
        p.patent_date,
        a.disambig_assignee_organization as organization
    FROM patents_2021 p
    INNER JOIN ai_patents ai ON p.patent_id = ai.patent_id
    INNER JOIN biopharma_patents bp ON p.patent_id = bp.patent_id
    LEFT JOIN g_assignee_disambiguated a ON p.patent_id = a.patent_id
    WHERE a.assignee_type IN ('2', '3')  -- Companies only
    LIMIT 10
""").df()

print("Sample Biopharma AI Patents (2021):")
print(biopharma_ai)

Sample Biopharma AI Patents (2021):
Empty DataFrame
Columns: [patent_id, patent_title, patent_date, organization]
Index: []


### Validation Strategy

**To ensure classification quality:**

1. **Manual Review Sample**
   - Review 50-100 patents from each method
   - Calculate precision = (true positives) / (total identified)
   
2. **Compare to Known AI Patents**
   - Use publications citing use of ML/AI
   - Cross-reference with companies' AI patent portfolios
   
3. **Temporal Validation**
   - AI patent counts should increase over time
   - Check for breaks/anomalies in time series

4. **Cross-Database Check**
   - Compare PatentsView results to Google Patents searches
   - Spot-check specific patent IDs in USPTO records

## Summary and Next Steps

### Key Takeaways:

1. **Data Access:** PatentsView bulk downloads are superior to USPTO for research
   - Pre-processed, disambiguated
   - Research-ready relational structure
   - ~9-10 GB for full dataset

2. **Import Process:** DuckDB handles large files efficiently
   - 2021 subset: ~350k patents
   - 4 core tables imported successfully
   - ~2 GB database for 2021 only

3. **Data Structure:** Well-documented, clear relationships
   - `patent_id` is universal join key
   - `assignee_id` enables firm-level aggregation
   - CPC codes standardized across patents

4. **AI Classification:** Dual approach recommended
   - **Primary:** CPC codes (G06N family)
   - **Secondary:** Keyword validation
   - **Biopharma:** Combine with A61/C07/C12 CPC sections

### Recommended Pipeline for Full Analysis:

```python
# Pseudocode for firm-year AI patent metrics

1. Identify AI patents:
   - Filter g_cpc_current for G06N% codes
   - Optional: supplement with keyword search
   
2. Filter to biopharma:
   - Join with g_cpc_current for A61/C07/C12
   - OR filter assignees to known biopharma firms
   
3. Map to firms:
   - Join g_assignee_disambiguated on patent_id
   - Group by assignee_id (not organization name!)
   
4. Aggregate by year:
   - Extract year from patent_date
   - COUNT(DISTINCT patent_id) per firm-year
   
5. Calculate metrics:
   - AI patent count
   - AI patent share (AI / total patents)
   - AI patent intensity (AI patents / R&D spend)
```

### Files Delivered:
1. ✅ `task1_patentsview_exploration.ipynb` - This notebook
2. ✅ `patentsview_2021.ddb` - DuckDB database with 2021 data
3. ✅ Data dictionary documentation (embedded above)
4. ✅ AI classification strategy (CPC + keywords)

### References:
- PatentsView Download Page: https://patentsview.org/download/data-download-tables
- PatentsView API Docs: https://search.patentsview.org/docs/
- CPC Code Lookup: https://www.cooperativepatentclassification.org/
- USPTO Patent Database: https://www.uspto.gov/patents/search